In [13]:
import pandas as pd
from datetime import datetime

In [14]:
df_balance = pd.read_csv('data/electricitybalance.csv',delimiter=';')
# source: https://www.energidataservice.dk/dataset/electricitybalance

In [15]:
df_capacities = pd.read_csv('data/capacitypermunicipality.csv',delimiter=';')
# source: https://www.energidataservice.dk/dataset/capacitypermunicipality/resource_extract/fa9e8860-a055-4bd4-aca3-260b7ad23564?page=7#resource-preview

In [20]:
df_balance

,HourUTC,HourDK,PriceArea,GrossCon,NetCon,LocalPowerProd,OffshoreWindPower,OnshoreWindPower,CentralProd,ElectricBoilerCon,SolarPowerProd,ExchangeContinent,ExchangeGreatBelt,ExchangeNordicCountries,WindCapacityFactor
0,2020-04-03 21:00,2020-04-03 23:00,DK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-03 20:00,2020-04-03 22:00,DK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-03 19:00,2020-04-03 21:00,DK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-03 18:00,2020-04-03 20:00,DK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-03 17:00,2020-04-03 19:00,DK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81136,2011-01-01 03:00,2011-01-01 04:00,DK1,1904.2,1818.1,428.7,609.550003,1749.849997,886.2,NaN,NaN,-361.5,-300.5,-1108.1,0.476674
81137,2011-01-01 02:00,2011-01-01 03:00,DK1,1950.0,1863.2,429.2,613.623262,1744.176738,880.8,NaN,NaN,-366.5,-330.3,-1020.9,0.475129
81138,2011-01-01 01:00,2011-01-01 02:00,DK1,2033.9,1946.7,431.4,615.024796,1760.675204,917.0,NaN,NaN,-354.0,-297.0,-1039.3,0.479623
81139,2011-01-01 00:00,2011-01-01 01:00,DK1,2129.5,2040.2,436.4,621.428703,1883.371297,1083.2,NaN,NaN,-434.3,-309.0,-1151.6,0.513047


## Wind capacity factor

In [21]:
# Municipalities with number higher than 400 are in DK1
dk1_municipality_seperator = 400
filter = [a and b for a,b in zip(df_capacities['Month']=='2019-07-01 00:00',df_capacities['MunicipalityNo']>dk1_municipality_seperator)]
wind_capacity = df_capacities.iloc[filter]['OnshoreWindCapacity'].sum()

In [22]:
df_balance['WindCapacityFactor'] = df_balance['OnshoreWindPower']/wind_capacity

In [24]:
filter = [a and b for a,b in zip( df_balance['HourDK'] > '2019-01-01 00:00', df_balance['HourDK'] < '2020-01-01 00:00')]
df_balance.loc[filter]['WindCapacityFactor'].mean()

0.24669173859751334

## Solar capacity factor

In [25]:
# Municipalities with number higher than 400 are in DK1
dk1_municipality_seperator = 400
filter = [a and b for a,b in zip(df_capacities['Month']=='2019-12-01 00:00',df_capacities['MunicipalityNo']>dk1_municipality_seperator)]
solar_capacity = df_capacities.iloc[filter]['SolarPowerCapacity'].sum()
solar_capacity

738.0768999999999

In [27]:
solar_capacity = (662+738) / 2

In [28]:
df_balance['SolarCapacityFactor'] = df_balance['SolarPowerProd']/solar_capacity

In [29]:
filter = [a and b for a,b in zip( df_balance['HourDK'] > '2019-01-01 00:00', df_balance['HourDK'] < '2020-01-01 00:00')]
df_balance.loc[filter]['SolarCapacityFactor'].mean()

0.10187550973803544

## Creating new data frame 

In [30]:
df_capfactor = pd.DataFrame(index=df_balance['HourUTC'])
df_capfactor['HourDK'] = df_balance['HourDK'].values
df_capfactor['WindCapacityFactor'] = df_balance['WindCapacityFactor'].values
df_capfactor['SolarCapacityFactor'] = df_balance['SolarCapacityFactor'].values

In [31]:
df_capfactor.to_csv('data/windcapacityfacotor.csv')

In [32]:
df_capfactor

,HourDK,WindCapacityFactor,SolarCapacityFactor
HourUTC,,,
2020-04-03 21:00,2020-04-03 23:00,NaN,NaN
2020-04-03 20:00,2020-04-03 22:00,NaN,NaN
2020-04-03 19:00,2020-04-03 21:00,NaN,NaN
2020-04-03 18:00,2020-04-03 20:00,NaN,NaN
2020-04-03 17:00,2020-04-03 19:00,NaN,NaN
...,...,...,...
2011-01-01 03:00,2011-01-01 04:00,0.476674,NaN
2011-01-01 02:00,2011-01-01 03:00,0.475129,NaN
2011-01-01 01:00,2011-01-01 02:00,0.479623,NaN
